In [63]:
# Libraries Used 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_datareader import data as wb

# Magic
%matplotlib inline

In [64]:
# Collect historical prices by Yahoo! Finance est. 2000 
tickers = ['AAPL', 'JNJ', 'TSLA', 'LMND', 'AGNC', 'IBM', 'XOM', 'BRK-B', 'PFE', 'AMZN']
start_date = '2000-01-01'
historical_data = pd.DataFrame()
for t in tickers:
    historical_data[t] = wb.DataReader(t, data_source='yahoo', start=start_date)['Adj Close']

In [65]:
# Remove all rows that includes NaN values
# Check if all columns match and are float variables
historical_data = historical_data.dropna()
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 174 entries, 2020-07-02 to 2021-03-11
Data columns (total 10 columns):
AAPL     174 non-null float64
JNJ      174 non-null float64
TSLA     174 non-null float64
LMND     174 non-null float64
AGNC     174 non-null float64
IBM      174 non-null float64
XOM      174 non-null float64
BRK-B    174 non-null float64
PFE      174 non-null float64
AMZN     174 non-null float64
dtypes: float64(10)
memory usage: 15.0 KB


In [66]:
historical_data.head()

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
Date,,,,,,,,,,
2020-07-02,90.571838,138.209732,241.731995,69.410004,11.923519,114.928406,41.504036,178.830002,31.749561,2890.300049
2020-07-06,92.994652,140.180374,274.316010,81.190002,12.120370,115.398880,41.795918,182.720001,31.749561,3057.040039
2020-07-07,92.706093,140.052917,277.971985,78.790001,12.017258,112.892921,40.713127,181.149994,31.307959,3000.120117
2020-07-08,94.865234,140.464706,273.175995,68.510002,12.176613,113.017738,40.618969,181.199997,31.050354,3081.110107
2020-07-09,95.273193,139.699966,278.855988,77.010002,11.876652,111.097466,38.942989,178.800003,30.783550,3182.629883


In [67]:
historical_data.tail()

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
Date,,,,,,,,,,
2021-03-05,121.419998,156.100006,597.950012,92.900002,15.900000,122.830002,60.930000,253.149994,34.389999,3000.459961
2021-03-08,116.360001,157.399994,563.000000,86.750000,16.330000,124.809998,60.869999,257.609985,34.349998,2951.949951
2021-03-09,121.089996,157.699997,673.580017,93.519997,16.350000,124.180000,59.930000,259.019989,34.450001,3062.850098
2021-03-10,119.980003,159.149994,668.059998,94.070000,16.400000,127.870003,61.770000,263.989990,34.930000,3057.639893
2021-03-11,121.959999,159.139999,699.599976,105.879997,16.309999,127.139999,61.310001,260.910004,34.709999,3113.590088


In [68]:
# Calculate log returns of each companies 
# Check if all columns match and are float variables
returns = np.log(historical_data/historical_data.shift(1))
returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 174 entries, 2020-07-02 to 2021-03-11
Data columns (total 10 columns):
AAPL     173 non-null float64
JNJ      173 non-null float64
TSLA     173 non-null float64
LMND     173 non-null float64
AGNC     173 non-null float64
IBM      173 non-null float64
XOM      173 non-null float64
BRK-B    173 non-null float64
PFE      173 non-null float64
AMZN     173 non-null float64
dtypes: float64(10)
memory usage: 15.0 KB


In [69]:
returns.head()

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
Date,,,,,,,,,,
2020-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-06,0.026399,0.014158,0.126451,0.156761,0.016375,0.004085,0.007008,0.021519,0.000000,0.056087
2020-07-07,-0.003108,-0.000910,0.013240,-0.030006,-0.008544,-0.021955,-0.026248,-0.008630,-0.014007,-0.018795
2020-07-08,0.023023,0.002936,-0.017404,-0.139806,0.013173,0.001105,-0.002315,0.000276,-0.008262,0.026638
2020-07-09,0.004291,-0.005459,0.020579,0.116956,-0.024943,-0.017137,-0.042136,-0.013333,-0.008630,0.032418


In [70]:
returns.tail()

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
Date,,,,,,,,,,
2021-03-05,0.010681,0.019601,-0.038532,-0.030739,-0.001257,0.022393,0.037116,0.030767,0.005540,0.007658
2021-03-08,-0.042567,0.008293,-0.060228,-0.068493,0.026685,0.015991,-0.000985,0.017465,-0.001164,-0.016300
2021-03-09,0.039845,0.001904,0.179327,0.075145,0.001224,-0.005060,-0.015563,0.005458,0.002907,0.036880
2021-03-10,-0.009209,0.009153,-0.008229,0.005864,0.003053,0.029282,0.030241,0.019006,0.013837,-0.001703
2021-03-11,0.016368,-0.000063,0.046131,0.118267,-0.005503,-0.005725,-0.007475,-0.011736,-0.006318,0.018133


In [71]:
# Create a table for Daily and Annual Risk Returns 
# Daily mean formula: daily_mean = returns.mean()
# Daily stdv formula: daily_mean = returns.std()
# Annual mean formula: annual_mean = returns.mean()*250
# Annual stdv formula: annual_mean = returns.std()*250 ** .5

daily_mean = []
daily_stdv = []

annual_mean = []
annual_stdv = []

for t in tickers:
    daily_mean.append(round(returns[t].mean()*100, 2))
    daily_stdv.append(round(returns[t].std()*100, 2))
    
    annual_mean.append(round((returns[t].mean()*250)*100, 2))
    annual_stdv.append(round((returns[t].std()*250**.5)*100, 2))

daily_mean = np.array(daily_mean)
daily_stdv = np.array(daily_stdv)
annual_mean = np.array(annual_mean)
annual_stdv = np.array(annual_stdv)
annual_stdv

array([ 38.5 ,  17.44,  78.93, 103.26,  17.56,  27.53,  39.87,  19.01,
        24.64,  34.15])

In [72]:
# Create DataFrame for returns and risk 
returns_data = pd.DataFrame({'Symbol':tickers, 
                             'Daily Average Return (%)':daily_mean,
                             'Daily Risk (%)':daily_stdv,
                             'Annual Average Return (%)':annual_mean,
                             'Annual Risk (%)':annual_stdv
                            })
returns_data

,Symbol,Daily Average Return (%),Daily Risk (%),Annual Average Return (%),Annual Risk (%)
0,AAPL,0.17,2.44,43.00,38.50
1,JNJ,0.08,1.10,20.38,17.44
2,TSLA,0.61,4.99,153.57,78.93
3,LMND,0.24,6.53,61.02,103.26
4,AGNC,0.18,1.11,45.27,17.56
5,IBM,0.06,1.74,14.59,27.53
6,XOM,0.23,2.52,56.38,39.87
7,BRK-B,0.22,1.20,54.59,19.01
8,PFE,0.05,1.56,12.88,24.64
9,AMZN,0.04,2.16,10.75,34.15


In [73]:
# Returns the lowest return in history between 09-30-2014 to now
returns.idxmin() 

AAPL    2020-09-03
JNJ     2021-01-29
TSLA    2020-09-08
LMND    2021-03-02
AGNC    2021-02-25
IBM     2021-01-22
XOM     2020-11-30
BRK-B   2020-10-28
PFE     2020-10-28
AMZN    2020-10-30
dtype: datetime64[ns]

In [74]:
# Returns the highest return in history between 09-30-2014 to now
returns.idxmax()

AAPL    2020-07-31
JNJ     2021-01-26
TSLA    2021-03-09
LMND    2021-01-07
AGNC    2020-11-10
IBM     2020-10-08
XOM     2020-11-09
BRK-B   2020-11-09
PFE     2020-11-09
AMZN    2020-07-20
dtype: datetime64[ns]

In [75]:
# Covariance daily
# Determines if there is a relationship between two stocks, whether they move together:
# positively, negatively, or neutral 
return_cov = returns.cov()
return_cov

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
AAPL,0.000593,0.000067,0.000606,0.000180,0.000048,0.000026,0.000004,0.000057,0.000033,0.000372
JNJ,0.000067,0.000122,0.000107,0.000076,0.000030,0.000068,0.000084,0.000073,0.000073,0.000037
TSLA,0.000606,0.000107,0.002492,0.000885,0.000073,0.000067,0.000004,0.000058,0.000084,0.000517
LMND,0.000180,0.000076,0.000885,0.004265,0.000071,0.000119,0.000223,0.000134,0.000079,0.000175
AGNC,0.000048,0.000030,0.000073,0.000071,0.000123,0.000063,0.000105,0.000063,0.000018,0.000021
IBM,0.000026,0.000068,0.000067,0.000119,0.000063,0.000303,0.000207,0.000103,0.000074,0.000008
XOM,0.000004,0.000084,0.000004,0.000223,0.000105,0.000207,0.000636,0.000180,0.000093,-0.000055
BRK-B,0.000057,0.000073,0.000058,0.000134,0.000063,0.000103,0.000180,0.000145,0.000062,0.000022
PFE,0.000033,0.000073,0.000084,0.000079,0.000018,0.000074,0.000093,0.000062,0.000243,-0.000005
AMZN,0.000372,0.000037,0.000517,0.000175,0.000021,0.000008,-0.000055,0.000022,-0.000005,0.000467


In [ ]:
# Covariance yearly
# Determines if there is a relationship between two stocks, whether they move together:
# positively, negatively, or neutral 
return_cov = returns.cov()*250
return_cov

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
AAPL,0.148236,0.016745,0.151432,0.044967,0.011886,0.006457,0.001108,0.014327,0.008185,0.093017
JNJ,0.016745,0.030421,0.026821,0.018923,0.007389,0.017109,0.021092,0.018153,0.018331,0.009151
TSLA,0.151432,0.026821,0.623030,0.221354,0.018254,0.016842,0.000913,0.014489,0.021002,0.129318
LMND,0.044967,0.018923,0.221354,1.066216,0.017829,0.029629,0.055851,0.033492,0.019763,0.043729
AGNC,0.011886,0.007389,0.018254,0.017829,0.030853,0.015819,0.026354,0.015867,0.004468,0.005346
IBM,0.006457,0.017109,0.016842,0.029629,0.015819,0.075777,0.051846,0.025818,0.018413,0.002009
XOM,0.001108,0.021092,0.000913,0.055851,0.026354,0.051846,0.158927,0.045078,0.023328,-0.013634
BRK-B,0.014327,0.018153,0.014489,0.033492,0.015867,0.025818,0.045078,0.036151,0.015599,0.005375
PFE,0.008185,0.018331,0.021002,0.019763,0.004468,0.018413,0.023328,0.015599,0.060693,-0.001354
AMZN,0.093017,0.009151,0.129318,0.043729,0.005346,0.002009,-0.013634,0.005375,-0.001354,0.116637


In [ ]:
# Correlations
# Determines how close the relationship are between two stocks
return_corr = returns.corr()
return_corr

,AAPL,JNJ,TSLA,LMND,AGNC,IBM,XOM,BRK-B,PFE,AMZN
AAPL,1.000000,0.249352,0.498296,0.113108,0.175761,0.060919,0.007218,0.195718,0.086294,0.707404
JNJ,0.249352,1.000000,0.194817,0.105070,0.241192,0.356334,0.303334,0.547400,0.426613,0.153617
TSLA,0.498296,0.194817,1.000000,0.271588,0.131659,0.077512,0.002901,0.096542,0.108000,0.479719
LMND,0.113108,0.105070,0.271588,1.000000,0.098301,0.104236,0.135678,0.170590,0.077689,0.124003
AGNC,0.175761,0.241192,0.131659,0.098301,1.000000,0.327171,0.376364,0.475104,0.103259,0.089119
IBM,0.060919,0.356334,0.077512,0.104236,0.327171,1.000000,0.472437,0.493277,0.271510,0.021366
XOM,0.007218,0.303334,0.002901,0.135678,0.376364,0.472437,1.000000,0.594717,0.237526,-0.100141
BRK-B,0.195718,0.547400,0.096542,0.170590,0.475104,0.493277,0.594717,1.000000,0.333012,0.082777
PFE,0.086294,0.426613,0.108000,0.077689,0.103259,0.271510,0.237526,0.333012,1.000000,-0.016094
AMZN,0.707404,0.153617,0.479719,0.124003,0.089119,0.021366,-0.100141,0.082777,-0.016094,1.000000


In [ ]:
no_assets = len(tickers)

In [ ]:
loop = 1000

In [ ]:
pflio_ret = []
pflio_vol = [] 
pflio_wei = []

for x in range(loop):
    weights = np.random.random(no_assets)
    weights /= np.sum(weights)
    
    pflio_wei.append(weights)
    pflio_ret.append(np.sum(weights*returns.mean())*250)
    pflio_vol.append(np.sqrt(np.dot(weights.T, np.dot(returns.cov()*250, weights))))
    #print(x, weights)

pflio_wei = np.array(pflio_wei)
pflio_ret = np.array(pflio_ret)
pflio_vol = np.array(pflio_vol)

print(np.sum(weights))
#pflio_returns, pflio_volatilities

In [ ]:
pflio_scenarios = pd.DataFrame({'Return': pflio_ret, 'Volatility': pflio_vol})
#pflio_scenarios = pflio_scenarios.sort_values('Return', ascending=True)
pflio_scenarios.to_csv("ret_vol_index.csv")

In [ ]:
pflio_scenarios.head()
#pflio_scenarios['Return']

In [ ]:
pflio_scenarios.tail()

In [ ]:
for i in range(loop):
    if pflio_scenarios['Return'][i] >= 0.70 and pflio_scenarios['Volatility'][i] <= 0.55:
        print(pflio_scenarios.iloc[i])

In [ ]:
index = 376
pfolio_wei2 = []
tickers2 = []
tickers2 = np.array(tickers)
pflio_wei2 = pflio_wei[index]

print(pflio_scenarios.iloc[index])
tickers_df = pd.DataFrame({'Symbol':tickers})

weights_df = pd.DataFrame({'Weights': pflio_wei2})
ideal_portfolio = pd.concat([tickers_df, weights_df], axis=1)
ideal_portfolio = ideal_portfolio.sort_values('Weights', ascending=False)
ideal_portfolio

In [ ]:
# Calculate Weighted Returns 
weights = np.array([pflio_wei2])
weighted_returns = ( weights * returns)

In [ ]:
#pflio_wei2

In [ ]:
capital = 18000
price = []

print(pflio_scenarios.iloc[index])

for i in range(len(pflio_wei2)):
    price.append(round(capital*pflio_wei2[i],2))
price = np.array(price)
price_portfolio = pd.DataFrame({'Price':price})
ideal_portfolio = pd.concat([tickers_df, price_portfolio, weights_df], axis=1)
ideal_portfolio = ideal_portfolio.sort_values('Price', ascending=False)
ideal_portfolio

In [ ]:
# Calculating Diversifiable and Non-Diversifiable

# Portfolio Return
portfolio_return = weighted_returns.sum(axis=1)
print("Portfolio Return (10 days)", portfolio_return.mean()*10)
print("Portfolio Return (250 days)", portfolio_return.mean()*250)
print("Portfolio Return (756 days)", portfolio_return.mean()*756)

print("")

# Portfolio variance
pflio_var = np.sqrt(np.dot(pflio_wei2.T, np.dot(returns.cov()*10, pflio_wei2)))
print("Portfolio Volatility (10 days)", pflio_var)
pflio_var = np.sqrt(np.dot(pflio_wei2.T, np.dot(returns.cov()*250, pflio_wei2)))
print("Portfolio Volatility (250 days)", pflio_var)
pflio_var = np.sqrt(np.dot(pflio_wei2.T, np.dot(returns.cov()*756, pflio_wei2)))
print("Portfolio Volatility (756 days)", pflio_var)

# print("Portfolio Variance", pflio_var)

# Portfolio volatility 
#pflio_vol = (np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight'])))**.5
#print("Portfolio Volatility:", pflio_vol)

# or...

#pflio_vol2 = np.sqrt(pflio_var)
#print("Portfolio Volatility:", pflio_vol2)


# Calculating variance annually for each tickers
var_a = []
for t in tickers:
    var_a.append(returns[t].var()*250)

var_a = np.array(var_a)
variances = pd.DataFrame({'Symbol':tickers, 'Variance':var_a})
print("")
# Diversifiable Risk
# div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0] - ...
#                  ... - my_portfolio['Weight'][n]**2*variances['Variance'][n]
div_risk = 0   
for i in range(len(tickers_df)):
    if i==0:
        div_risk = pflio_var - pflio_wei2[0]**2*variances['Variance'][0]
    else:
        div_risk -= pflio_wei2[i]**2*variances['Variance'][i]
print("Diversifiable Risk:", round(div_risk,2)*100, "%")

print("")
# Non Diversifiable Risk
non_div_risk1 = 0
non_div_risk1 = pflio_var - div_risk
non_div_risk1 = round(non_div_risk1, 4)*100
print("Non-Diversifiable Risk:", non_div_risk1, "%")

#print("")
non_div_risk2 = 0
for i in range(len(tickers_df)):
    non_div_risk2 += pflio_wei2[i]**2*variances['Variance'][i]

    
print("")
non_div_risk2 = round(non_div_risk2, 4)*100
print("Non-Diversifiable Risk:", non_div_risk2, "%")
print("Non-Diversifiable Risk:", non_div_risk2 == non_div_risk1)

In [ ]:
# Efficient Frontier 
pflio_scenarios.plot(x='Volatility', y='Return', kind='scatter', figsize=(10,6));
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')

In [ ]:
(historical_data/historical_data.iloc[0]*100).plot(figsize=(20,15))

In [ ]:
#sns.set_style('whitegrid')
#regression = sns.pairplot(returns[1:], kind="reg")
#regression

In [ ]:
# Excatly -1: Perfect negative relationship 
# -0.70:      Strong negative relationship
# -0.50:      Moderate negative relationship
# -0.30:      Weak negative relationship
#  0.00:      No correlation
# +0.30:      Weak positive relationship
# +0.50:      Moderate positive relationship
# +0.70:      Strong positive relationship
# Exactly +1: Perfect positive relationship

plt.subplots(figsize=(15, 15))
heatmap = sns.heatmap(return_corr, annot=True, square=True, cmap='coolwarm')
heatmap
